# 0. 라이브러리

In [112]:
import pandas as pd
import numpy as np
import time

import OpenDartReader

In [113]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [114]:
api_key = '462c1cb331fdef6dbc18da2fc8253c2b774e925f'
dart = OpenDartReader(api_key)

- `finstate(corp, bsns_year, reprt_code)`
    - corp: 기업명
    - bsns_year: 사업연도
    - reprt_code: 보고서 유형 11011: 사업보고서, '11012': 반기보고서, '11013': 1분기보고서, '11014: 3분기보고서)
- 이 메서드는 아래 컬럼을 가지는 데이터프레임을 반환합니다.
    - account_nm: 계정명 ex. 자본총계, 영업이익, 당기순이익 등
    - fs_nm: 개별/연결명 ex. 연결재무제표 또는 재무제표
    - sj_nm: 재무제표명 ex. 재무상태표 또는 손익계산서
    - thstrm_dt: 당기일자  ex. 2019.12.31 현재, 2019.01.01 ~ 2019.12.31
    - thstrm_amount: 당기금액 ex. 166,009,000,000
- 계정명과 당기/전기/전전기 금액 컬럼을 바탕으로 우리가 원하는 값을 가져올 수 있습니다

In [115]:
result = dart.finstate("비트나인", 2022, "11011") # 2020년 래몽래인 사업보고서 내 재무 정보
result = result.loc[result.fs_nm == "연결재무제표"] # 연결재무제표 필터링

# account_nm이 당기순이익, 영업이익, 매출액 중 하나인 경우에만 가져오기
result = result.loc[result.account_nm.isin(['당기순이익', '영업이익', '매출액'])]

result = result.loc[:, ['account_nm', 'fs_nm', 'sj_nm', 'thstrm_dt', 'thstrm_amount']]
display(result)

,account_nm,fs_nm,sj_nm,thstrm_dt,thstrm_amount
9,영업이익,연결재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"1,620,879,293"
10,당기순이익,연결재무제표,손익계산서,2022.01.01 ~ 2022.12.31,"-528,858,705"


# 1. 재무변수 수집 함수

In [116]:
def find_financial_ind(corp_nm, yr, inds):
    report = dart.finstate(corp_nm, yr) # 데이터 가져오기

    if report is None:
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[corp_nm, yr] + [np.nan] * len(inds)]
        data = [[corp_nm, yr-1] + [np.nan] * len(inds)]
        data = [[corp_nm, yr-2] + [np.nan] * len(inds)]
        return pd.DataFrame(data, columns=['기업명', '연도'] + inds)
    
    else:
        report = report[report.account_nm.isin(inds)]
        if sum(report.fs_nm == '연결재무제표') > 0:
            # 연결재무제표 데이터가 있으면 연결재무제표 사용
            report = report.loc[report.fs_nm == '연결재무제표']

        else:
            # 연결재무제표 데이터가 없으면 일반재무제표 사용
            report = report.loc[report.fs_nm == '재무제표']

        data = []
        for y, c in zip([yr, yr-1, yr-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [corp_nm, y]
            for ind in inds:
                # account_nm이 ind인 행의 c 컬럼 값을 가져 옴
                if sum(report.account_nm == ind) > 0:
                    value = report.loc[report.account_nm == ind, c].iloc[0]
                else:
                    value = np.nan

                record.append(value)
            
            data.append(record)

        return pd.DataFrame(data, columns=['기업명', '연도'] + inds)

In [117]:
def str_to_float(value):
    if type(value) == float:
        return value
    elif value == '-':
        return 0
    else:
        return float(value.replace(',', ''))

In [118]:
inds = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익', '유동자산', '유동부채', '이자비용']
display(find_financial_ind('삼성전자', 2020, inds))

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동자산,유동부채,이자비용
0,삼성전자,2020,"378,235,718,000,000","102,287,702,000,000","275,948,016,000,000","236,806,988,000,000","35,993,876,000,000","26,407,832,000,000","198,215,579,000,000","75,604,351,000,000",NaN
1,삼성전자,2019,"352,564,497,000,000","89,684,076,000,000","262,880,421,000,000","230,400,881,000,000","27,768,509,000,000","21,738,865,000,000","181,385,260,000,000","63,782,764,000,000",NaN
2,삼성전자,2018,"339,357,244,000,000","91,604,067,000,000","247,753,177,000,000","243,771,415,000,000","58,886,669,000,000","44,344,857,000,000","174,697,424,000,000","69,081,510,000,000",NaN


# 2. 부도기업 재무 변수 수집

In [119]:
delist_corp = pd.read_csv('./data/부도기업.csv', dtype={'종목코드':str})
delist_corp['폐지일'] = pd.to_datetime(delist_corp['폐지일'])

delist_codes = delist_corp.종목코드.to_list()
delist_yrs = delist_corp.폐지일.dt.year.to_list()

delist_codes[:3], delist_yrs[:3]

(['069110', '078650', '096640'], [2023, 2023, 2023])

In [120]:
len(delist_codes), len(delist_yrs)

(141, 141)

In [122]:
inds = ['자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익',\
    '유동부채', '유동자산', '비유동자산', '비유동부채']

delist_data = pd.DataFrame()

for idx, (corp_nm, yr) in enumerate(zip(delist_codes, delist_yrs)):
    print(idx+1, "/", len(delist_codes))
    try:
        yr -= 1
        result = find_financial_ind(corp_nm, yr, inds)
    except:
        pass

    delist_data = pd.concat([delist_data, result], axis=0, ignore_index=True)
    time.sleep(0.5)

for ind in inds:
    delist_data[ind] = delist_data[ind].apply(str_to_float)

1 / 141
2 / 141
3 / 141
4 / 141
5 / 141
6 / 141
7 / 141
8 / 141
9 / 141
10 / 141
11 / 141
12 / 141
13 / 141
14 / 141
15 / 141
16 / 141
17 / 141
18 / 141
19 / 141
20 / 141
21 / 141
22 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

23 / 141
24 / 141
25 / 141
26 / 141
27 / 141
28 / 141
29 / 141
30 / 141
31 / 141
32 / 141
33 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

34 / 141
35 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

36 / 141
37 / 141
38 / 141
39 / 141
40 / 141
41 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

42 / 141
43 / 141
44 / 141
45 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

46 / 141
47 / 141
48 / 141
49 / 141
50 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

51 / 141
52 / 141
53 / 141
54 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

55 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

56 / 141
57 / 141
58 / 141
59 / 141
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

60 / 141
61 / 141
62 / 141
63 / 141
64 / 141
6

In [123]:
delist_data_raw = delist_data.copy()
delist_data_raw.to_csv('./data/부도기업재무원본.csv', index=False)

In [124]:
delist_data.shape

(423, 12)

In [125]:
delist_data.sort_values(by=['기업명', '연도'], inplace=True)
delist_data.drop_duplicates(inplace=True)

In [126]:
delist_data.head(3)

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채
113,008800,2018,1.061340e+11,4.557505e+10,6.055897e+10,8.673759e+09,-1.547890e+10,-1.094641e+10,4.158662e+10,5.587494e+10,5.025908e+10,3.988427e+09
112,008800,2019,3.888764e+10,1.399830e+10,2.488934e+10,4.929243e+09,-4.661418e+09,-4.175835e+10,1.204562e+10,1.121310e+10,2.767454e+10,1.952681e+09
111,008800,2020,3.199762e+10,1.282887e+10,1.916875e+10,8.087514e+09,-2.933185e+09,-4.336539e+09,1.125048e+10,8.297428e+09,2.370019e+10,1.578396e+09


In [127]:
delist_data.isnull().sum()

기업명      0
연도       0
자산총계     0
부채총계     0
자본총계     3
매출액      3
영업이익     0
당기순이익    0
유동부채     3
유동자산     3
비유동자산    3
비유동부채    3
dtype: int64

In [128]:
delist_data.shape

(222, 12)

# 3. 결측치 knn

In [129]:
from sklearn.impute import KNNImputer

null_cols = delist_data.iloc[:, 1:].columns

imputer=KNNImputer(n_neighbors=5)
filled_delist_data = imputer.fit_transform(delist_data.iloc[:, 1:])
filled_delist_data = pd.DataFrame(filled_delist_data, columns=delist_data.iloc[:, 1:].columns)

In [130]:
filled_delist_data.shape, delist_data.shape

((222, 11), (222, 12))

In [131]:
filled_delist_data.isnull().sum()

연도       0
자산총계     0
부채총계     0
자본총계     0
매출액      0
영업이익     0
당기순이익    0
유동부채     0
유동자산     0
비유동자산    0
비유동부채    0
dtype: int64

In [132]:
delist_data = delist_data.reset_index().drop(['index'], axis=1)
delist_data[null_cols] = filled_delist_data

In [133]:
delist_data.shape

(222, 12)

In [134]:
delist_data.isnull().sum()

기업명      0
연도       0
자산총계     0
부채총계     0
자본총계     0
매출액      0
영업이익     0
당기순이익    0
유동부채     0
유동자산     0
비유동자산    0
비유동부채    0
dtype: int64

# 4. 재무 비율 변수 구성
- https://gils-lab.tistory.com/38
- https://dacon.io/competitions/official/235946/codeshare/5805
- https://blog.naver.com/o12486vs2/222096044791

### (1) 건전성

In [135]:
# 부채비율 = 총부채 / 총자산 DR (LEV)
delist_data['부채비율'] = delist_data['부채총계'] / delist_data['자본총계'] * 100

# 자기자본비율 DER
delist_data['자기자본비율'] = delist_data['부채총계'] / delist_data['자산총계'] * 100

# 유동부채비율
delist_data['유동부채비율'] = delist_data['유동부채'] / delist_data['자본총계'] * 100

# 유동부채비율
delist_data['비유동부채비율'] = delist_data['비유동부채'] / delist_data['자본총계'] * 100


### (2) 수익성

In [136]:
# 총자산영업이익율 OI/TA
delist_data['총자산영업이익율'] = delist_data['영업이익'] / delist_data['자산총계']

# 총자산순이익율 ROA
delist_data['ROA'] = delist_data['당기순이익'] / delist_data['자산총계']

# 자기자본이익율 ROE
avg_eq = delist_data['자본총계'].rolling(2).mean()
delist_data['ROE'] = delist_data['당기순이익'] / avg_eq
# delist_data.loc[delist_data.연도 == 2020, 'ROE'] = np.nan

# 매출액영업이익율 OPM
delist_data['매출액영업이익율'] = delist_data['영업이익'] / delist_data['매출액']

# 매출액순이익율
delist_data['매출액순이익율'] = delist_data['당기순이익'] / delist_data['매출액']


### (3) 성장성

In [137]:
delist_data['총자산증가율'] = delist_data['자산총계'].diff() / delist_data['자산총계'] * 100
delist_data.loc[delist_data.연도 == 2020, '총자산증가율'] = np.nan

delist_data['매출액증가율'] = delist_data['매출액'].diff() / delist_data['매출액'] * 100
delist_data.loc[delist_data.연도 == 2020, '매출액증가율'] = np.nan

delist_data['당기순이익증가율'] = delist_data['당기순이익'].diff() / delist_data['당기순이익'] * 100
delist_data.loc[delist_data.연도 == 2020, '당기순이익증가율'] = np.nan

delist_data['영업이익증가율'] = delist_data['영업이익'].diff() / delist_data['영업이익'] * 100
delist_data.loc[delist_data.연도 == 2020, '영업이익증가율'] = np.nan

### (4) 유동성

In [138]:
# 유동비율 LIQ
delist_data['유동비율'] = delist_data['유동자산'] / delist_data['유동부채']

### (5) 활동성

In [139]:
# 자산회전율
delist_data['자산회전율'] = delist_data['매출액'] / delist_data['자산총계']

# 부채회전율
delist_data['부채회전율'] = delist_data['매출액'] / delist_data['부채총계']

# 자본회전율
delist_data['자본회전율'] = delist_data['매출액'] / delist_data['자본총계']


### (6) 규모

In [140]:
# 총매출액규모
delist_data['총매출액규모'] = np.log(delist_data['매출액'])

# 총매출액규모
delist_data['총자산규모'] = np.log(delist_data['자산총계'])

/Users/yeorii/opt/anaconda3/envs/yeonii/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/yeorii/opt/anaconda3/envs/yeonii/lib/python3.9/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [141]:
delist_data.head()

,기업명,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,008800,2018.0,1.061340e+11,4.557505e+10,6.055897e+10,8.673759e+09,-1.547890e+10,-1.094641e+10,4.158662e+10,5.587494e+10,5.025908e+10,3.988427e+09,75.257303,42.941037,68.671281,6.586022,-0.145843,-0.103138,NaN,-1.784566,-1.262014,NaN,NaN,NaN,NaN,1.343580,0.081725,0.190318,0.143228,22.883568,25.387968
1,008800,2019.0,3.888764e+10,1.399830e+10,2.488934e+10,4.929243e+09,-4.661418e+09,-4.175835e+10,1.204562e+10,1.121310e+10,2.767454e+10,1.952681e+09,56.242161,35.996789,48.396711,7.845450,-0.119869,-1.073821,-0.977394,-0.945666,-8.471555,-172.924782,-75.965347,73.786306,-232.064112,0.930886,0.126756,0.352131,0.198046,22.318451,24.383942
2,008800,2020.0,3.199762e+10,1.282887e+10,1.916875e+10,8.087514e+09,-2.933185e+09,-4.336539e+09,1.125048e+10,8.297428e+09,2.370019e+10,1.578396e+09,66.925972,40.093205,58.691757,8.234215,-0.091669,-0.135527,-0.196856,-0.362681,-0.536202,NaN,NaN,NaN,NaN,0.737518,0.252754,0.630415,0.421911,22.813587,24.188927
3,017680,2016.0,5.303433e+10,1.856882e+10,3.446551e+10,5.539841e+10,-8.128229e+09,-8.678103e+09,1.737932e+10,3.648390e+10,1.655043e+10,1.189500e+09,53.876518,35.012826,50.425243,3.451275,-0.153264,-0.163632,-0.323603,-0.146723,-0.156649,39.666209,85.401182,50.028953,63.913599,2.099271,1.044576,2.983411,1.607358,24.737817,24.694205
4,017680,2017.0,5.603977e+10,2.720401e+10,2.883576e+10,4.684759e+10,-9.783131e+09,-2.129312e+10,2.633363e+10,3.091101e+10,2.512876e+10,8.703753e+08,94.341229,48.544115,91.322840,3.018389,-0.174575,-0.379964,-0.672755,-0.208829,-0.454519,5.363040,-18.252435,59.244568,16.915869,1.173822,0.835970,1.722084,1.624635,24.570165,24.749327


In [142]:
delist_data.shape

(222, 31)

In [143]:
delist_data.isnull().sum()

기업명          0
연도           0
자산총계         0
부채총계         0
자본총계         0
매출액          0
영업이익         0
당기순이익        0
유동부채         0
유동자산         0
비유동자산        0
비유동부채        0
부채비율         2
자기자본비율       2
유동부채비율       2
비유동부채비율      2
총자산영업이익율     2
ROA          2
ROE          1
매출액영업이익율     3
매출액순이익율      3
총자산증가율      39
매출액증가율      39
당기순이익증가율    39
영업이익증가율     39
유동비율         2
자산회전율        2
부채회전율        2
자본회전율        2
총매출액규모       0
총자산규모        0
dtype: int64

In [144]:
delist_rows = np.arange(2, len(delist_data), 3)
delist_data = delist_data.iloc[delist_rows, :]

In [151]:
delist_data = delist_data.reset_index().drop(['index'], axis=1)
delist_data.rename(columns={'기업명':'종목코드'}, inplace=True)
delist_data.head()

,종목코드,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,008800,2020.0,3.199762e+10,1.282887e+10,1.916875e+10,8.087514e+09,-2.933185e+09,-4.336539e+09,1.125048e+10,8.297428e+09,2.370019e+10,1.578396e+09,66.925972,40.093205,58.691757,8.234215,-0.091669,-0.135527,-0.196856,-0.362681,-0.536202,NaN,NaN,NaN,NaN,0.737518,0.252754,0.630415,0.421911,22.813587,24.188927
1,017680,2018.0,6.653430e+10,3.649968e+10,3.003462e+10,5.162126e+10,-6.876169e+09,-1.057761e+10,3.528285e+10,3.706857e+10,2.055660e+10,1.216835e+09,121.525364,54.858442,117.473923,4.051441,-0.103348,-0.158980,-0.359353,-0.133204,-0.204908,15.773120,9.247483,-101.303722,-42.275902,1.050612,0.775859,1.414293,1.718725,24.667199,24.920983
2,023430,2016.0,3.448916e+10,2.518239e+10,9.306768e+09,1.211601e+10,-3.751118e+09,-1.421161e+11,8.108206e+09,6.565781e+09,2.792337e+10,1.707418e+10,270.581460,73.015380,87.121615,183.459844,-0.108762,-4.120602,-4.350440,-0.309600,-11.729612,-85.620040,-3.283316,91.294977,43.884732,0.809770,0.351299,0.481130,1.301849,23.217793,24.263911
3,026260,2017.0,5.171141e+10,2.212413e+10,2.958728e+10,1.198680e+10,-1.174278e+10,-1.283146e+10,1.465260e+10,2.708373e+10,2.462768e+10,7.471526e+09,74.775798,42.783840,49.523307,25.252491,-0.227083,-0.248136,-0.808883,-0.979642,-1.070466,24.280479,-155.490696,-285.724968,-12.145574,1.848391,0.231802,0.541798,0.405133,23.207072,24.668944
4,030270,2019.0,4.586739e+10,2.748884e+10,1.837855e+10,2.550810e+09,-3.967914e+09,-1.452529e+10,1.975572e+10,2.156882e+10,2.379226e+10,7.733123e+09,149.570231,59.931119,107.493338,42.076893,-0.086508,-0.316680,-0.643280,-1.555551,-5.694384,-34.430538,-457.385441,-137.487113,-161.813750,1.091776,0.055613,0.092794,0.138793,21.659677,24.549020


In [152]:
delist_data.to_csv('./data/부도기업재무.csv', index=False)

# 실패한 애들 알아보기

In [158]:
delist_corp = pd.read_csv('./data/부도기업.csv', dtype={'종목코드':str})
print(delist_corp.shape)
delist_corp.head()

(141, 5)


,기업명,종목코드,폐지일,상장일,지속기간
0,코스온,069110,2023-10-20,2003-10-14,7311 days
1,지나인제약,078650,2023-07-21,2010-10-22,4655 days
2,멜파스,096640,2023-07-17,2009-12-18,4959 days
3,엠피씨플러스,050540,2023-05-08,2005-12-12,6356 days
4,제이웨이,058420,2023-04-14,2002-01-10,7764 days


In [154]:
delist_done = pd.read_csv('./data/부도기업재무.csv', dtype={'종목코드':str})
print(delist_done.shape)
delist_done.head()

(74, 31)


,종목코드,연도,자산총계,부채총계,자본총계,매출액,영업이익,당기순이익,유동부채,유동자산,비유동자산,비유동부채,부채비율,자기자본비율,유동부채비율,비유동부채비율,총자산영업이익율,ROA,ROE,매출액영업이익율,매출액순이익율,총자산증가율,매출액증가율,당기순이익증가율,영업이익증가율,유동비율,자산회전율,부채회전율,자본회전율,총매출액규모,총자산규모
0,008800,2020.0,3.199762e+10,1.282887e+10,1.916875e+10,8.087514e+09,-2.933185e+09,-4.336539e+09,1.125048e+10,8.297428e+09,2.370019e+10,1.578396e+09,66.925972,40.093205,58.691757,8.234215,-0.091669,-0.135527,-0.196856,-0.362681,-0.536202,NaN,NaN,NaN,NaN,0.737518,0.252754,0.630415,0.421911,22.813587,24.188927
1,017680,2018.0,6.653430e+10,3.649968e+10,3.003462e+10,5.162126e+10,-6.876169e+09,-1.057761e+10,3.528285e+10,3.706857e+10,2.055660e+10,1.216835e+09,121.525364,54.858442,117.473923,4.051441,-0.103348,-0.158980,-0.359353,-0.133204,-0.204908,15.773120,9.247483,-101.303722,-42.275902,1.050612,0.775859,1.414293,1.718725,24.667199,24.920983
2,023430,2016.0,3.448916e+10,2.518239e+10,9.306768e+09,1.211601e+10,-3.751118e+09,-1.421161e+11,8.108206e+09,6.565781e+09,2.792337e+10,1.707418e+10,270.581460,73.015380,87.121615,183.459844,-0.108762,-4.120602,-4.350440,-0.309600,-11.729612,-85.620040,-3.283316,91.294977,43.884732,0.809770,0.351299,0.481130,1.301849,23.217793,24.263911
3,026260,2017.0,5.171141e+10,2.212413e+10,2.958728e+10,1.198680e+10,-1.174278e+10,-1.283146e+10,1.465260e+10,2.708373e+10,2.462768e+10,7.471526e+09,74.775798,42.783840,49.523307,25.252491,-0.227083,-0.248136,-0.808883,-0.979642,-1.070466,24.280479,-155.490696,-285.724968,-12.145574,1.848391,0.231802,0.541798,0.405133,23.207072,24.668944
4,030270,2019.0,4.586739e+10,2.748884e+10,1.837855e+10,2.550810e+09,-3.967914e+09,-1.452529e+10,1.975572e+10,2.156882e+10,2.379226e+10,7.733123e+09,149.570231,59.931119,107.493338,42.076893,-0.086508,-0.316680,-0.643280,-1.555551,-5.694384,-34.430538,-457.385441,-137.487113,-161.813750,1.091776,0.055613,0.092794,0.138793,21.659677,24.549020


In [164]:
delist_corps_done = delist_done.종목코드

delist_corps_fail = delist_corp[~delist_corp.종목코드.isin(delist_corps_done)]
print(delist_corps_fail.shape)
delist_corps_fail.head()

(67, 5)


,기업명,종목코드,폐지일,상장일,지속기간
21,자안바이오,221610,2022-01-05,2016-01-27,2170 days
32,럭슬,033600,2021-08-11,2000-11-21,7568 days
34,에스앤씨엔진그룹,900080,2021-06-14,2009-12-04,4210 days
40,미래SCI,028040,2021-04-13,1996-07-27,9026 days
44,바이오빌,065940,2020-07-31,2003-01-07,6415 days


In [165]:
delist_corps_fail.to_csv('./data/부도기업재무실패.csv', index=False)